# Aula 06 - Filtragem Baseada em Conhecimento - Exercícios

## Importação dos dados (MovieLens 100k)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import wget
# !python3 -m wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
# !tar -xvzf ml-100k.zip


In [3]:
#Types of genres
genre = pd.read_csv('./ml-100k/u.genre', sep="|", encoding='latin-1', header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [4]:
#Types of occupations
occupation = pd.read_csv('./ml-100k/u.occupation', sep="|", encoding='latin-1', header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [5]:


#Load the Ratings data
data = pd.read_csv('./ml-100k/u.data', sep="\t", header=None)
data.columns = ['userId', 'movieId', 'rating', 'timestamp']
data.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
#Load the Movies data
item = pd.read_csv('./ml-100k/u.item', sep="|", encoding='latin-1', header=None)
item.columns = ['movieId', 'title' ,'release','video release date', 'IMDb URL', 'unknown', 'Action', 
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item['release'] = pd.to_datetime(item['release'])
item = item[pd.notnull(item['release'])]
item['year'] = item['release'].dt.year.astype(int)
item.drop(columns=['release', 'video release date', 'IMDb URL'], inplace=True)
item.head()

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995


In [7]:
df_meta = item.melt(id_vars=['movieId', 'title'], var_name='genre')
df_meta = df_meta[df_meta.value == 1]
df_meta.drop(columns=['value'], inplace=True)
# df_meta[df_meta['movieId']==1]
df_meta.head()

,movieId,title,genre
1371,1373,Good Morning (1971),unknown
1682,2,GoldenEye (1995),Action
1684,4,Get Shorty (1995),Action
1697,17,From Dusk Till Dawn (1996),Action
1701,21,Muppet Treasure Island (1996),Action


In [8]:
# Obter a lista de gêneros de um item
def get_genres(df, movieId):
    if movieId not in df['movieId'].values:
        return []
    return df.loc[(df.movieId==movieId),'genre'].tolist()

get_genres(df_meta, 1)

['Animation', "Children's", 'Comedy']

In [9]:


#Load the User data
user = pd.read_csv('./ml-100k/u.user', sep="|", encoding='latin-1', header=None)
user.columns = ['userId', 'age', 'gender', 'occupation', 'zip code']
user.head()

,userId,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


### Obs. Nesta aula, você poderá optar por resolver o exercício 1 OU o exercício 2. 

***Exercício 01:*** Implemente um sistema de recomendação baseado em casos que permite as seguintes restrições definidas pelo usuário:
- Escolher entre filmes mais antigos ou mais recentes
- Determinar os gêneors de maior preferência (mas não restrito a eles)
- Escolher os filmes mais populares ou os menos populares dentre os usuários de uma dada profissão.
- Atribuir importância maior ou menor para cada uma das restrições anteriores.

Recomende uma lista de 10 filmes com base nos critérios definidos pelo usuário.

Exemplo: usuário define:
- filmes_mais_recentes = True
- generos_preferidos = ['Drama']
- filmes_mais_populares = True
- profissão = 'writer'
- pesos = [0.33, 0.33, 0.33]

Calculando w_year

In [10]:
def set_w_year(df, ctrl):
    df_copy = df.copy()

    if ctrl is None:
        df_copy['w_year'] = -1
        return df_copy

    min_year = df_copy['year'].min()
    max_year = df_copy['year'].max()
    normalized = (df_copy['year'] - min_year) / (max_year - min_year)

    if(ctrl):
        df_copy['w_year'] = normalized
    else:
        df_copy['w_year'] = 1-normalized

    return df_copy

Calculando w_genre

In [11]:
generos_preferidos = ['War', 'Drama']

def set_w_genres(df, generos_pref):
    df_copy = df.copy()
    
    if generos_pref is None:
        df_copy['w_genre'] = -1
        return df_copy
        
    for index, row in df_copy.iterrows():
        # Get the genres for the current movie
        movie_id = row['movieId']
        movie_genres = get_genres(df_meta, movie_id)
        
        score = len(list(set(movie_genres) & set(generos_pref))) / len(set(movie_genres + generos_pref))
        df_copy.loc[index, 'w_genre'] = score
        
    return df_copy

Calculando w_popularity (popularidade por profissao)

Pré calculo da popularidade de todos os filmes por todas as profissoes

In [12]:

merged_df = pd.merge(data, user, on='userId')

occupation_popularity = merged_df.groupby(['occupation', 'movieId']).size().reset_index(name='qtt_ratings')
occupation_popularity = pd.merge(occupation_popularity, item[['movieId', 'title']], on='movieId')

# Normalizar a popularidade para cada profissão usando o Min-Max
max_qtt = occupation_popularity.groupby('occupation')['qtt_ratings'].transform('max')
min_qtt = occupation_popularity.groupby('occupation')['qtt_ratings'].transform('min')

occupation_popularity['w_popularity'] = (occupation_popularity['qtt_ratings'] - min_qtt) / (max_qtt - min_qtt)
    

In [13]:
def set_w_popularity(df, profissao, mais_populares):
    df_copy = df.copy()

    if profissao is None or mais_populares is None:
        df_copy['w_popularity'] = -1
        return df_copy
    
    occ_scores = occupation_popularity[occupation_popularity['occupation'] == profissao]
    score_lookup = pd.Series(occ_scores.w_popularity.values, index=occ_scores.movieId).to_dict()
    
    for index, row in df_copy.iterrows():
        movie_id = row['movieId']
        
        score = score_lookup.get(movie_id, 0.0)
        
        if not mais_populares:
            score = 1 - score
        
        df_copy.loc[index, 'w_popularity'] = score
        
    return df_copy

In [14]:
import numpy as np

def knowledge_content_based(
    filmes_mais_recentes=None,
    generos_preferidos=None,
    filmes_mais_populares=None,
    profissao=None,
    pesos = [0.33, 0.33, 0.33] ):
    
    df = item.copy()

    #Usando as funções anteriores para calcular a relevancia para os critérios fornecidos
    df = set_w_year(df, filmes_mais_recentes)
    df = set_w_genres(df, generos_preferidos)
    df = set_w_popularity(df, profissao, filmes_mais_populares)


    # Initialize total score to zero
    df['total_score'] = 0.0
    
    # Create a dictionary of the scores and their corresponding weights
    df['w_year'] = df['w_year'] * (pesos[0] if df['w_year'].iloc[0] != -1 else 0)
    df['w_genre'] = df['w_genre'] * (pesos[1] if df['w_genre'].iloc[0] != -1 else 0)
    df['w_popularity'] = df['w_popularity'] * (pesos[2] if df['w_popularity'].iloc[0] != -1 else 0)
    
    df['w'] = df['w_year'] + df['w_genre'] + df['w_popularity']
    
    result = df.sort_values(by='w', ascending=False)
    
    return result[['title', 'year', 'w']].head(10)

# Exemplo do código funcionadno

In [15]:

# Definindo preferencias do usuário para o exemplo
filmes_mais_recentes_pref = False # Prefers older movies
generos_preferidos_pref = ['War', 'Drama']
profissao_pref = 'writer'
filmes_mais_populares_pref = True # Prefers most popular
pesos_pref = [0.2, 0.5, 0.3] # [year, genre, popularity]

# Get the final recommendations!
top10 = knowledge_content_based(
    pesos=pesos_pref,
    filmes_mais_recentes=filmes_mais_recentes_pref,
    generos_preferidos=generos_preferidos_pref,
    profissao=profissao_pref,
    filmes_mais_populares=filmes_mais_populares_pref
)

print("TOP 10 RECOMENDAÇÕES")
display(top10)

# --- Example with one criterion turned OFF ---
print("\nEXEMPLO IGNORANDO O ANO")
top10_no_year = knowledge_content_based(
    pesos=[0.0, 0.5, 0.5],
    generos_preferidos=generos_preferidos_pref,
    profissao=profissao_pref,
    filmes_mais_populares=filmes_mais_populares_pref
)
display(top10_no_year)

TOP 10 RECOMENDAÇÕES


,title,year,w
198,"Bridge on the River Kwai, The (1957)",1957,0.723279
179,Apocalypse Now (1979),1979,0.676923
204,Patton (1970),1970,0.642915
189,Henry V (1989),1989,0.615992
285,"English Patient, The (1996)",1996,0.615520
520,"Deer Hunter, The (1978)",1978,0.610324
640,Paths of Glory (1957),1957,0.607895
317,Schindler's List (1993),1993,0.605466
482,Casablanca (1942),1942,0.596086
646,Ran (1985),1985,0.580364



EXEMPLO IGNORANDO O ANO


,title,year,w
285,"English Patient, The (1996)",1996,0.794872
179,Apocalypse Now (1979),1979,0.711538
198,"Bridge on the River Kwai, The (1957)",1957,0.692308
317,Schindler's List (1993),1993,0.653846
189,Henry V (1989),1989,0.653846
99,Fargo (1996),1997,0.625000
204,Patton (1970),1970,0.615385
257,Contact (1997),1997,0.608974
520,"Deer Hunter, The (1978)",1978,0.596154
470,Courage Under Fire (1996),1996,0.596154


***Exercício 02:*** Considere um CSP definido como uma tripla (V, D, C) onde:
- V = V_usuario U V_filme : variáveis relacionadas com o usuário e o filme, respectivamente
- C = C_usuario U C_filme U C_ui: restrições do usuário, filme e compatibilidade usuário/filme
- D = conjunto de domínios finitos para as variáveis

Podemos definir as variáveis e restrições como:
- V_usuario : {companhia(sozinho, casal, familia), filme_popular(sim, nao), filme_classico(sim, nao)}
- V_filme : {ano_lancamento(int), generos(list), n_avaliacoes(int)}
- C_usuario : {companhia=casal -> filme_popular=sim ^ companhia=sozinho -> filme_classico=nao}
- C_filme : {lista de itens do catalogo}
- C_ui : {companhia=casal -> generos=[Romance, Drama, Comedy], companhia=familia -> generos=[Children's], filme_classico=sim -> ano_lancamento < 1990, filme_classico=nao -> ano_lancamento > 1993, filme_popular=sim -> n_avaliacoes > 100, companhia=sozinho -> generos=[Horror, War]}

Dada a requisição: REQ={companhia=sozinho, filme_popular=sim}, recomende uma lista de 10 filmes para o usuário.

In [16]:
# TODO